In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import datetime, gc, random
import json, numpy as np, pandas as pd, zipfile
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.cluster import MeanShift, estimate_bandwidth
# file
import zipfile

# Data preprocessing
import datetime
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# Visualization
from matplotlib import pyplot as plt
import folium
import seaborn as sns
sns.set_theme(style="ticks", color_codes=True)
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from colorama import Fore

# Random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor

# parameter opimization
from sklearn.model_selection import GridSearchCV

# Validation
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%%time
# Defining all our palette colours.
primary_blue = "#496595"
primary_blue2 = "#85a1c1"
primary_blue3 = "#3f4d63"
primary_grey = "#c6ccd8"
primary_black = "#202022"
primary_bgcolor = "#f4f0ea"

primary_green = px.colors.qualitative.Plotly[2]

plt.rcParams['axes.facecolor'] = primary_bgcolor

colors = [primary_blue, primary_blue2, primary_blue3, primary_grey, primary_black, primary_bgcolor, primary_green]
sns.palplot(sns.color_palette(colors))

# 1. Loading the Data

In [ ]:
%%time
# Reading The Data

# sample
zip_file = zipfile.ZipFile("/kaggle/input/pkdd-15-predict-taxi-service-trajectory-i/sampleSubmission.csv.zip")
sample = pd.read_csv(zip_file.open('sampleSubmission.csv'))

# train
zip_file = zipfile.ZipFile("/kaggle/input/pkdd-15-predict-taxi-service-trajectory-i/train.csv.zip")
train = pd.read_csv(zip_file.open("train.csv"))

# test
zip_file = zipfile.ZipFile("/kaggle/input/pkdd-15-predict-taxi-service-trajectory-i/test.csv.zip")
test = pd.read_csv(zip_file.open("test.csv"))

# location_Data
zip_file = zipfile.ZipFile("/kaggle/input/pkdd-15-predict-taxi-service-trajectory-i/metaData_taxistandsID_name_GPSlocation.csv.zip")
location = pd.read_csv(zip_file.open("metaData_taxistandsID_name_GPSlocation.csv"))

Data Description

In [ ]:
%%time
sample

In [ ]:
%%time
train

In [ ]:
%%time
test

In [ ]:
%%time
location

# 2. Exploratory Data Analysis

In [ ]:
%%time
# Time data preprocessing
train["TIMESTAMP"] = [float(time) for time in train["TIMESTAMP"]]
train["data_time"] = [datetime.datetime.fromtimestamp(time, datetime.timezone.utc) for time in train["TIMESTAMP"]]

In [ ]:
# Time data
train["data_time"].value_counts()

In [ ]:
%%time
# Time data preparation
train["year"] = train["data_time"].dt.year
train["month"] = train["data_time"].dt.month
train["day"] = train["data_time"].dt.day
train["hour"] = train["data_time"].dt.hour
train["min"] = train["data_time"].dt.minute
train["weekday"] = train["data_time"].dt.weekday

In [ ]:
%%time
train.head()

In [ ]:
%%time
train.tail()

Time series visualization

In [ ]:
%%time
# Time series visualization
pivot = pd.pivot_table(train, index='month', columns="year", values="TRIP_ID", aggfunc="count").reset_index()

# Visualization, per month count
with plt.style.context("ggplot"):
    plt.figure(figsize=(10,6))
    plt.rcParams["font.size"] = 18
    plt.plot(pivot["month"], pivot[2013], label="2013")
    plt.plot(pivot["month"], pivot[2014], label="2014")
    plt.xlabel("month")
    plt.ylabel("count")
    plt.legend(facecolor="white")

In [ ]:
%%time
# Time series visualization
pivot = pd.pivot_table(train, index='day', columns="month", values="TRIP_ID", aggfunc="count").reset_index()
# Visualization, per day count
with plt.style.context("fivethirtyeight"):
    plt.figure(figsize=(10,6))
    plt.rcParams["font.size"] = 18
    plt.plot(pivot["day"], pivot[1], label="January")
    plt.plot(pivot["day"], pivot[2], label="February")
    plt.xlabel("day")
    plt.ylabel("count")
    plt.legend(facecolor="white")

In [ ]:
%%time
# Visualization, per day count
with plt.style.context("fivethirtyeight"):
    plt.figure(figsize=(10,6))
    plt.rcParams["font.size"] = 18
    plt.plot(pivot["day"], pivot[3], label="March")
    plt.plot(pivot["day"], pivot[4], label="April")
    plt.xlabel("day")
    plt.ylabel("count")
    plt.legend(facecolor="white")

In [ ]:
%%time
# Visualization, per day count
with plt.style.context("fivethirtyeight"):
    plt.figure(figsize=(10,6))
    plt.rcParams["font.size"] = 18
    plt.plot(pivot["day"], pivot[5], label="May")
    plt.plot(pivot["day"], pivot[6], label="June")
    plt.xlabel("day")
    plt.ylabel("count")
    plt.legend(facecolor="white")

In [ ]:
%%time
# Visualization, per day count
with plt.style.context("fivethirtyeight"):
    plt.figure(figsize=(10,6))
    plt.rcParams["font.size"] = 18
    plt.plot(pivot["day"], pivot[7], label="July")
    plt.plot(pivot["day"], pivot[8], label="August")
    plt.xlabel("day")
    plt.ylabel("count")
    plt.legend(facecolor="white")

In [ ]:
%%time
# Visualization, per day count
with plt.style.context("fivethirtyeight"):
    plt.figure(figsize=(10,6))
    plt.rcParams["font.size"] = 18
    plt.plot(pivot["day"], pivot[9], label="September")
    plt.plot(pivot["day"], pivot[10], label="October")
    plt.xlabel("day")
    plt.ylabel("count")
    plt.legend(facecolor="white")

In [ ]:
%%time
# Visualization, per day count
with plt.style.context("fivethirtyeight"):
    plt.figure(figsize=(10,6))
    plt.rcParams["font.size"] = 18
    plt.plot(pivot["day"], pivot[11], label="November")
    plt.plot(pivot["day"], pivot[12], label="December")
    plt.xlabel("day")
    plt.ylabel("count")
    plt.legend(facecolor="white")

Call type

In [ ]:
%%time
### Call type
fig = px.histogram(train, x='CALL_TYPE', color='CALL_TYPE', color_discrete_sequence=[primary_blue, primary_grey],)
fig.update_layout(
    title_text='Call Type Distribution', # title of plot
    xaxis_title_text='Call Type', # xaxis label
    yaxis_title_text='Count', # yaxis label
    bargap=0.2, # gap between bars of adjacent location coordinates
    paper_bgcolor=primary_bgcolor,
    plot_bgcolor=primary_bgcolor,
)
fig.show()

In [ ]:
%%time
# Checking Missing Values
nan_data = (train.isna().sum().sort_values(ascending=False) / len(train) * 100)[:9]
fig, ax = plt.subplots(1,1,figsize=(12, 5))

ax.bar(nan_data.index, 100, color=primary_grey, width=0.6)

bar = ax.bar(
    nan_data.index, 
    nan_data, 
    color=primary_blue, 
    width=0.6
)
ax.bar_label(bar, fmt='%.01f %%')
ax.spines.left.set_visible(False)
ax.set_yticks([])
ax.set_title('Missing value Ratio', fontweight='bold')

plt.show()

Sampling 1st & last ride coordinate and last coordinate

# 3. Data Preprocessing

Training set preprocessing

In [ ]:
%%time
# First longitude
lists_1st_lon = []
for i in range(0,len(train["POLYLINE"])):
    if train["POLYLINE"][i] == '[]':
        k=0
        lists_1st_lon.append(k)
    else:
        k = re.sub(r"[[|[|]|]|]]", "", train["POLYLINE"][i]).split(",")[0]
        lists_1st_lon.append(k)
        
train["lon_1st"] = lists_1st_lon

# First latitude
lists_1st_lat = []
for i in range(0,len(train["POLYLINE"])):
    if train["POLYLINE"][i] == '[]':
        k=0
        lists_1st_lat.append(k)
    else:
        k = re.sub(r"[[|[|]|]|]]", "", train["POLYLINE"][i]).split(",")[1]
        lists_1st_lat.append(k)
        
train["lat_1st"] = lists_1st_lat

In [ ]:
%%time
# Last longitude
lists_last_lon = []
for i in range(0,len(train["POLYLINE"])):
        if train["POLYLINE"][i] == '[]':
            k=0
            lists_last_lon.append(k)
        else:
            k = re.sub(r"[[|[|]|]|]]", "", train["POLYLINE"][i]).split(",")[-2]
            lists_last_lon.append(k)

train["lon_last"] = lists_last_lon

# Last latitude
lists_last_lat = []
for i in range(0,len(train["POLYLINE"])):
    if train["POLYLINE"][i] == '[]':
        k=0
        lists_last_lat.append(k)
    else:
        k = re.sub(r"[[|[|]|]|]]", "", train["POLYLINE"][i]).split(",")[-1]
        lists_last_lat.append(k)
        
train["lat_last"] = lists_last_lat

In [ ]:
%%time
# Delete lon & lat have "0".
train = train.query("lon_last != 0")

In [ ]:
%%time
train["lon_1st"] = [float(k) for k in train["lon_1st"]]
train["lat_1st"] = [float(k) for k in train["lat_1st"]]
train["lon_last"] = [float(k) for k in train["lon_last"]]
train["lat_last"] = [float(k) for k in train["lat_last"]]

In [ ]:
%%time
# Visualization, sampling 5000 datas.
mapping_1st = pd.DataFrame({
    "date":train.head(5000)["data_time"].values,
    "lat":train.head(5000)["lat_1st"].values,
    "lon":train.head(5000)["lon_1st"].values
})

mapping_last = pd.DataFrame({
    "date":train.head(5000)["data_time"].values,
    "lat":train.head(5000)["lat_last"].values,
    "lon":train.head(5000)["lon_last"].values
})

por_map = folium.Map(location=[41.141412,-8.590324], tiles='Stamen Terrain', zoom_start=11)

for i, r in mapping_1st.iterrows():
    folium.CircleMarker(location=[r["lat"],r["lon"]], radius=0.5, color="red").add_to(por_map)

for i, r in mapping_last.iterrows():
    folium.CircleMarker(location=[r["lat"],r["lon"]], radius=0.5, color="blue").add_to(por_map)    
    
por_map


Test set preprocessing

In [ ]:
%%time
# First longitude
lists_1st_lon = []
for i in range(0,len(test["POLYLINE"])):
    if test["POLYLINE"][i] == '[]':
        k=0
        lists_1st_lon.append(k)
    else:
        k = re.sub(r"[[|[|]|]|]]", "", test["POLYLINE"][i]).split(",")[0]
        lists_1st_lon.append(k)
        
test["lon_1st"] = lists_1st_lon

# First latitude
lists_1st_lat = []
for i in range(0,len(test["POLYLINE"])):
    if test["POLYLINE"][i] == '[]':
        k=0
        lists_1st_lat.append(k)
    else:
        k = re.sub(r"[[|[|]|]|]]", "", test["POLYLINE"][i]).split(",")[1]
        lists_1st_lat.append(k)
        
test["lat_1st"] = lists_1st_lat

In [ ]:
%%time
# Last longitude
lists_last_lon = []
for i in range(0,len(test["POLYLINE"])):
        if test["POLYLINE"][i] == '[]':
            k=0
            lists_last_lon.append(k)
        else:
            k = re.sub(r"[[|[|]|]|]]", "", test["POLYLINE"][i]).split(",")[-2]
            lists_last_lon.append(k)

test["lon_last"] = lists_last_lon

# Last latitude
lists_last_lat = []
for i in range(0,len(test["POLYLINE"])):
    if test["POLYLINE"][i] == '[]':
        k=0
        lists_last_lat.append(k)
    else:
        k = re.sub(r"[[|[|]|]|]]", "", test["POLYLINE"][i]).split(",")[-1]
        lists_last_lat.append(k)
        
test["lat_last"] = lists_last_lat

In [ ]:
%%time
# changin type str ⇒ float
test["lon_1st"] = [float(k) for k in test["lon_1st"]]
test["lat_1st"] = [float(k) for k in test["lat_1st"]]
test["lon_last"] = [float(k) for k in test["lon_last"]]
test["lat_last"] = [float(k) for k in test["lat_last"]]

Distribution of travel distance and direction

delta_lon = lon_last - lon_1st
delta_lat = lat_last - kat_last

In [ ]:
%%time

# Create delta parameter
train["delta_lon"] = train["lon_last"] - train["lon_1st"]
train["delta_lat"] = train["lat_last"] - train["lat_1st"]

test["delta_lon"] = test["lon_last"] - test["lon_1st"]
test["delta_lat"] = test["lat_last"] - test["lat_1st"]

In [ ]:
%%time
# sampling : 5,000 point
sample = train.head(5000)

with plt.style.context("fivethirtyeight"):
    fig, ax = plt.subplots(1,3,figsize=(20, 6))
    
    # plot
    ax[0].scatter(sample["delta_lon"], sample["delta_lat"], s=3, c="red")
    ax[0].set_xlabel("delta longitude")
    ax[0].set_ylabel("delta_latitude")
    
    # delta longitude distribution
    ax[1].hist(sample["delta_lon"], bins=30, color="red")
    ax[1].set_xlabel("delta longitude")
    ax[1].set_ylabel("count")
    ax[1].set_yscale("log")
    
    # delta latitude distribution
    ax[2].hist(sample["delta_lat"], bins=30, color="red")
    ax[2].set_xlabel("delta latitude")
    ax[2].set_ylabel("count")
    ax[2].set_yscale("log")

In [ ]:
%%time
# monthly, delta longtitude & latitude boxplot
train["month_str"] = [str(i) for i in train["month"]]

In [ ]:
%%time
with plt.style.context("fivethirtyeight"):
    fig, ax = plt.subplots(1,2, figsize=(20,6))
    # delta_lon
    sns.boxplot("month_str", "delta_lon", data=train, ax=ax[0])
    ax[0].set_xlabel("month")
    ax[0].set_ylabel("delta_lon")
    ax[0].set_ylim([-0.3,0.3])
    # delta_lat
    sns.boxplot("month_str", "delta_lat", data=train, ax=ax[1])
    ax[1].set_xlabel("month")
    ax[1].set_ylabel("delta_lon")
    ax[1].set_ylim([-0.3,0.3])

Weekday distribution

In [ ]:
%%time
# weekday, delta longtitude & latitude boxplot
train["weekday_str"] = [str(i) for i in train["weekday"]]

In [ ]:
%%time
with plt.style.context("fivethirtyeight"):
    fig, ax = plt.subplots(1,2, figsize=(20,6))
    # delta_lon
    sns.boxplot("weekday_str", "delta_lon", data=train, ax=ax[0])
    ax[0].set_xlabel("weekday\n (0:Monday ~ 6:Sunday)")
    ax[0].set_ylabel("delta_lon")
    ax[0].set_ylim([-0.3,0.3])
    # delta_lat
    sns.boxplot("weekday_str", "delta_lat", data=train, ax=ax[1])
    ax[1].set_xlabel("weekday\n (0:Monday ~ 6:Sunday)")
    ax[1].set_ylabel("delta_lon")
    ax[1].set_ylim([-0.3,0.3])

Call type : A: 1, B: 2, C:3

In [ ]:
%%time
# copy dataframe
df_ml = train.copy()

# outlier is dropped
df_ml = df_ml.query("delta_lon <= 0.2 & delta_lon >= -0.2 & delta_lat <= 0.2 & delta_lat >= -0.2")



In [ ]:
%%time
# Call type <= 0.2 & delta_lon>=
map_call = {"A":1, "B":2, "C":3}
df_ml["Call_type"] = df_ml["CALL_TYPE"].map(map_call)

# Origin_call
def origin_call_flg(x):
    if x["ORIGIN_CALL"] == None:
        res = 0
    else:
        res = 1
    return res
df_ml["ORIGIN_CALL"] = df_ml.apply(origin_call_flg, axis=1)

# Origin_stand
def origin_stand_flg(x):
    if x["ORIGIN_STAND"] == None:
        res = 0
    else:
        res=1
    return res
df_ml["ORIGIN_STAND"] = df_ml.apply(origin_stand_flg, axis=1)

# Day type
df_ml.drop("DAY_TYPE", axis=1, inplace=True)

# Missing data
def miss_flg(x):
    if x["MISSING_DATA"] == "False":
        res = 0
    else:
        res = 1
    return res
df_ml["MISSING_DATA"] = df_ml.apply(miss_flg, axis=1)

In [ ]:
%%time
df_ml = df_ml.sample(50000)

In [ ]:
%%time
X = df_ml[["Call_type", 'ORIGIN_CALL', 'ORIGIN_STAND', 'MISSING_DATA', 'lon_1st', 'lat_1st', 'delta_lon', 'delta_lat']]

In [ ]:
%%time
y = df_ml[["lon_last","lat_last"]]

In [ ]:
%%time
# copy dataframe
df_ml_t = test.copy()

# Call type <= 0.2 & delta_lon>=
map_call = {"A":1, "B":2, "C":3}
df_ml_t["Call_type"] = df_ml_t["CALL_TYPE"].map(map_call)

# Origin_call
df_ml_t["ORIGIN_CALL"] = df_ml_t.apply(origin_call_flg, axis=1)

# Origin_stand
df_ml_t["ORIGIN_STAND"] = df_ml_t.apply(origin_stand_flg, axis=1)

# Day type
df_ml_t.drop("DAY_TYPE", axis=1, inplace=True)

# Missing data
df_ml_t["MISSING_DATA"] = df_ml_t.apply(miss_flg, axis=1)


In [ ]:
%%time
X_Test = df_ml_t[["Call_type", 'ORIGIN_CALL', 'ORIGIN_STAND', 'MISSING_DATA', 'lon_1st', 'lat_1st', 'delta_lon', 'delta_lat']]


train test split

In [ ]:
%%time
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# 4. Buliding Machine learning Models

 4.1 Random Forest Regressor

In [ ]:
%%time
forest = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=1))

# Fitting
forest = forest.fit(X_train, y_train)
y_train_pred = forest.predict(X_train)
y_test_pred = forest.predict(X_test)

# Make predictions on test data using the model trained on original data

# Performance metrics

print("Mean Square Error on training Data:{}".format(mean_squared_error(y_train, y_train_pred)))
print("Mean Square Error on testting Data:{}".format(mean_squared_error(y_test, y_test_pred)))
print("R2 score train:{}".format(r2_score(y_train, y_train_pred)))
print("R2 score test:{}".format(r2_score(y_test, y_test_pred)))

y_Test_pred = forest.predict(X_Test)

 4.2 Gradient Boosting Regressor

In [ ]:
bosting = MultiOutputRegressor(GradientBoostingRegressor(random_state=0))

# Fitting
bosting = bosting.fit(X_train, y_train)
y_train_pred2 = bosting.predict(X_train)
y_test_pred2 = bosting.predict(X_test)

# Evaluation
print("Mean Square Error on training Data:{}".format(mean_squared_error(y_train, y_train_pred2)))
print("Mean Square Error on testing Data:{}".format(mean_squared_error(y_test, y_test_pred2)))
print("R2 score train:{}".format(r2_score(y_train, y_train_pred2)))
print("R2 score test:{}".format(r2_score(y_test, y_test_pred2)))
y_Test_pred2 = bosting.predict(X_Test)
y_Test_pred2[2]

 4.3 Linear Regression for Multioutput Regression

In [ ]:
lr = MultiOutputRegressor(LinearRegression(n_jobs=1))

# Fitting
lr = lr.fit(X_train, y_train)
y_train_pred4 = lr.predict(X_train)
y_test_pred4 = lr.predict(X_test)

# Evaluation
print("Mean Square Error on training Data:{}".format(mean_squared_error(y_train, y_train_pred4)))
print("Mean Square Error on testing Data:{}".format(mean_squared_error(y_test, y_test_pred4)))
print("R2 score train:{}".format(r2_score(y_train, y_train_pred4)))
print("R2 score test:{}".format(r2_score(y_test, y_test_pred4)))
y_Test_pred4 = bosting.predict(X_Test)
y_Test_pred4[4]

 4.4 k-Nearest Neighbors for Multioutput Regression

In [ ]:
knn = MultiOutputRegressor(KNeighborsRegressor())

# Fitting
knn = knn.fit(X_train, y_train)
y_train_pred5 = knn.predict(X_train)
y_test_pred5 = knn.predict(X_test)

# Evaluation
print("Mean Square Error on training Data:{}".format(mean_squared_error(y_train, y_train_pred5)))
print("Mean Square Error on testing Data:{}".format(mean_squared_error(y_test, y_test_pred5)))
print("R2 score train:{}".format(r2_score(y_train, y_train_pred5)))
print("R2 score test:{}".format(r2_score(y_test, y_test_pred5)))
y_Test_pred5 = bosting.predict(X_Test)
y_Test_pred5[5]

 4.5 Decision Tree for Multioutput Regression

In [ ]:
dt = MultiOutputRegressor(DecisionTreeRegressor(max_depth=50, random_state=1))

# Fitting
dt = dt.fit(X_train, y_train)
y_train_pred6 = dt.predict(X_train)
y_test_pred6 = dt.predict(X_test)

# Evaluation
print("Mean Square Error on training Data:{}".format(mean_squared_error(y_train, y_train_pred6)))
print("Mean Square Error on testing Data:{}".format(mean_squared_error(y_test, y_test_pred6)))
print("R2 score train:{}".format(r2_score(y_train, y_train_pred6)))
print("R2 score test:{}".format(r2_score(y_test, y_test_pred6)))
y_Test_pred6 = bosting.predict(X_Test)
y_Test_pred6[5]

# 5. Submission

In [ ]:
%%time
submit_lat = y_Test_pred4.T[1]
submit_lon = y_Test_pred4.T[0]

In [ ]:
%%time
submit = pd.DataFrame({"TRIP_ID":test["TRIP_ID"], "LATITUDE":submit_lat, "LONGITUDE":submit_lon})

In [ ]:
submit.to_csv('submission.csv', index=False)
sub = pd.read_csv("./submission.csv")
sub.head(5)